# ToDo
* Höchstleistung in Variablenlimits
* Auch Entladen möglich? Dann muss der aktuelle Ladezustand zu jedem Zeitpunkt überwacht werden
* Skalare Parameter machen keinen Sinn, wenn konkrete Modelle initialisiert werden

# Assumptions and requirements
* Optimize electric car charging
* Neglection of battery lifetime saving charging
* Assuming charger has a maximum power. The actual charging power can be regulated between zero and the maximum charging power without intervals
* The optimization period can be of arbitray length. For the optimization period, a minimum and a maximum amount of charged energy can be defined as parameters.
* The time resolution is 15 minutes, according to the balancing time spans in the German wholesale market

# Model formulation

## Sets
Time steps - defining the start of each 15 minute long balancing interval

In [1]:
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint, Objective, minimize

model = AbstractModel()
model.time = Set(doc="Timesteps defining the start of each 15 minute long balancing interval")

## Parameters
Maximum charging power in kilowatts

In [2]:
model.maximum_power = Param(doc="Maximum charging power in kilowatts")

Electricity price in euros per megawatthour

In [3]:
model.electricity_price = Param(model.time, doc="Electricity price in Euro per Megawatthour for each balancing interval")

Minimum and maximum of charging energy in kilowatthours
* Presumably, the model will always decide to only charge the minimal amount. However in cases of very low, negative, electricity prices, charging more than required could be beneficial

In [4]:
model.minimum_energy = Param(doc="Minimum charging energy till last timestep")
model.maximum_energy = Param(doc="Maximum charging energy till last timestep")

## Variables
Charging power in kilowatts at each timestep

In [5]:
model.current_charging_power = Var(model.time, bounds=(0, None), doc="Charging power at each timestep in kilowatts")

## Constraints
Limit maximum charging power

In [6]:
def maximum_charging_power_rule(model, time):
    return model.current_charging_power[time] <= model.maximum_power
model.maximum_power_constraint = Constraint(model.time, rule=maximum_charging_power_rule, doc='Limit of maximum charging power')

Ensure that minimum energy is charged but maximum energy not exceeded

In [7]:
def minimum_energy_rule(model):
    return sum(model.current_charging_power[time] for time in model.time) * 0.25 >= model.minimum_energy
def maximum_energy_rule(model):
    return sum(model.current_charging_power[time] for time in model.time) * 0.25 <= model.maximum_energy

model.minimum_energy_constraint = Constraint(rule=minimum_energy_rule, doc='Ensure the minimum amount of electricity is charged')
model.maximum_energy_constraint = Constraint(rule=maximum_energy_rule, doc='Ensure the maximum amount of electricity is not exceeded')

## Objective

In [8]:
def objective_rule(model):
    return sum(model.electricity_price[time] * model.current_charging_power[time] for time in model.time)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Minimize electricity costs')
model.pprint()

1 Set Declarations
    time : Timesteps defining the start of each 15 minute long balancing interval
        Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        Not constructed

4 Param Declarations
    electricity_price : Electricity price in Euro per Megawatthour for each balancing interval
        Size=0, Index=time, Domain=Any, Default=None, Mutable=False
        Not constructed
    maximum_energy : Maximum charging energy till last timestep
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Not constructed
    maximum_power : Maximum charging power in kilowatts
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Not constructed
    minimum_energy : Minimum charging energy till last timestep
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Not constructed

1 Var Declarations
    current_charging_power : Charging power at each timestep in kilowatts
        Size=0, Index=time
        Not c

# Create and run the instance

In [9]:
instance = model.create_instance(filename='ev_charging.dat')
instance.pprint()

1 Set Declarations
    time : Timesteps defining the start of each 15 minute long balancing interval
        Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=(1, 4)
        [1, 2, 3, 4]

4 Param Declarations
    electricity_price : Electricity price in Euro per Megawatthour for each balancing interval
        Size=4, Index=time, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :    20
          2 :    30
          3 :   -20
          4 :    10
    maximum_energy : Maximum charging energy till last timestep
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     5
    maximum_power : Maximum charging power in kilowatts
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    22
    minimum_energy : Minimum charging energy till last timestep
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     3

1 Va